In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense

Using TensorFlow backend.
/Users/rkumar/anaconda2/envs/py3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:455: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/rkumar/anaconda2/envs/py3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:456: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/rkumar/anaconda2/envs/py3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:457: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/rkumar/anaconda2/en

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelEncoder_1 = LabelEncoder()
X[:,1] = labelEncoder_1.fit_transform(X[:,1])
labelEncoder_2 = LabelEncoder()
X[:,2] = labelEncoder_1.fit_transform(X[:,2])

In [5]:
ct = ColumnTransformer([('encoder', OneHotEncoder(), [1])], remainder='passthrough', sparse_threshold=0)
X = np.array(ct.fit_transform(X))
X = X[:, 1:]
X.shape

(10000, 11)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [7]:
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
scalar.fit(X_train)
X_train, X_test = scalar.transform(X_train), scalar.transform(X_test)

In [8]:
import tensorflow as tf
from keras import Sequential
from keras.layers import Dense
from keras.layers import Dropout
classifier = Sequential()
classifier.add(Dense(6, input_shape=(11,), activation = 'relu'))
# classifier.add(Dropout(rate=0.1))
classifier.add(Dense(6, activation = 'relu'))
# classifier.add(Dropout(rate=0.1))
classifier.add(Dense(1, activation = 'sigmoid'))
classifier.compile(optimizer=tf.train.AdagradOptimizer(0.01), loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, nb_epoch = 100)

/Users/rkumar/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/100
8000/8000 [==============================] - 1s 154us/step - loss: 0.5336 - acc: 0.7845
Epoch 2/100
8000/8000 [==============================] - 1s 131us/step - loss: 0.4728 - acc: 0.8052
Epoch 3/100
8000/8000 [==============================] - 1s 148us/step - loss: 0.4525 - acc: 0.8064 0s - loss: 0.4
Epoch 4/100
8000/8000 [==============================] - 1s 124us/step - loss: 0.4407 - acc: 0.8126
Epoch 5/100
8000/8000 [==============================] - 1s 130us/step - loss: 0.4325 - acc: 0.8174
Epoch 6/100
8000/8000 [==============================] - 1s 135us/step - loss: 0.4261 - acc: 0.8191
Epoch 7/100
8000/8000 [==============================] - 1s 155us/step - loss: 0.4208 - acc: 0.8216
Epoch 8/100
8000/8000 [==============================] - 1s 148us/step - loss: 0.4163 - acc: 0.8217
Epoch 9/100
8000/8000 [==============================] - 1s 130us/step - loss: 0.4123 - acc: 0.8236
Epoch 10/100
8000/8000 [==============================] - 1s 124us/step - loss: 0.408

In [10]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [11]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

In [12]:
mean

0.8518749944120646

In [ ]:
# Tuning the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [25, 32],
              'epochs': [100, 500],
              'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Epoch 1/100
7200/7200 [==============================] - 1s 84us/step - loss: 0.5465 - acc: 0.7971: 0s - loss: 0.6070 - acc: 0
Epoch 2/100
7200/7200 [==============================] - 0s 48us/step - loss: 0.4352 - acc: 0.7971
Epoch 3/100
7200/7200 [==============================] - 0s 55us/step - loss: 0.4300 - acc: 0.7971
Epoch 4/100
7200/7200 [==============================] - 0s 61us/step - loss: 0.4265 - acc: 0.7971
Epoch 5/100
7200/7200 [==============================] - 0s 59us/step - loss: 0.4234 - acc: 0.7971
Epoch 6/100
7200/7200 [==============================] - 0s 51us/step - loss: 0.4203 - acc: 0.7987: 0s - loss: 0.4200 - acc: 0.798
Epoch 7/100
7200/7200 [==============================] - 0s 50us/step - loss: 0.4185 - acc: 0.8215
Epoch 8/100
7200/7200 [==============================] - 0s 64us/step - loss: 0.4170 - acc: 0.8239
Epoch 9/100
7200/7200 [==============================] - 0s 51us/step - loss: 0.4155 - acc: 0.8275
Epoch 10/100
7200/7200 [=========================